In [3]:

# !pip install -qU datasets pinecone-client sentence-transformers torch
# !pip install PyPDF2
# !pip install fitz
# !pip install frontend
# !pip install tools
# !pip install --upgrade PyMuPDF
!pip install pytesseract
# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')

In [5]:
import fitz  # PyMuPDF
import io
from PIL import Image
import pytesseract
import pytesseract.pytesseract as pyt
import sys

# Path to the PDF file
pdf_file_path = "file4.pdf"

# Create a PDF document object
pdf_document = fitz.open(pdf_file_path)

# Initialize variables to store extracted text and images
pdf_text = ""
pdf_images = []

# Iterate through each page of the PDF
for page_num in range(len(pdf_document)):
    page = pdf_document[page_num]

    # Extract text from the page
    page_text = page.get_text()
    pdf_text += page_text

    # Extract images from the page
    images = page.get_images(full=True)
    for img_index, img in enumerate(images):
        xref = img[0]
        base_image = pdf_document.extract_image(xref)
        image_data = base_image["image"]

        # You can save the image if needed
        image = Image.open(io.BytesIO(image_data))
        image.save(f"image_{page_num}_{img_index}.jpg")

        # Append the image data to the list
        pdf_images.append(image_data)

# Close the PDF document
pdf_document.close()

# Extracted text and images are now available
print("Extracted Text:")
print(pdf_text)

# # If needed, you can extract text from images using Tesseract OCR
# extracted_text_from_images = []
# for image_data in pdf_images:
#     image = Image.open(io.BytesIO(image_data))
#     extracted_text = pytesseract.image_to_string(image, config="--psm 6")  # Modify config as needed
#     extracted_text_from_images.append(extracted_text)

# print("Extracted Text from Images:")
# for text in extracted_text_from_images:
#     print(text)

# You can further process the extracted text and images as needed.


Extracted Text:
Mr. Danny Le - New General Director of Masan Group (Source: Masan)
On June 19, Masan Group's Board of Directors passed a resolution to appoint Mr.
Danny Le to the position of General Director of Masan Group for a 5-year term.
According to research, Mr. Danny Le was born in 1984 and has a bachelor's
degree from Bowdoin University, USA.
Thursday, October 19, 2023 - 10:13 GMT+7
 RSS  Mobile
Follow us:    

ECONOMICS & DATA
DATA |  ADMINISTRATION |  BUSINESSES & ENTREPRENEURS
VPS is honored to receive the 2023
Outstanding Financial Brokerage
Service award
VPS became one of the ﬁrst 10
companies approved as a TPDNRL
trading member

LOG IN
New CEO Danny Le and Masan Group's desire to be a
"unicorn in the consumer industry".
Huy Binh
Saturday June 20, 2020 09:17 (GMT+7)

Facebook

Twitter

Zalo

Email

Copy the
link
Follow Viettimes
VietTimes – Mr. Danny Le plays an important role in building the growth
strategy, as well as directly organizing M&A transactions to 

In [8]:
#pdf_images

In [ ]:
import torch
from transformers import OscarModel, OscarTokenizer

# Load the Oscar model and tokenizer
model = OscarModel.from_pretrained("openai/clip-vit-base-patch16")
tokenizer = OscarTokenizer.from_pretrained("openai/clip-vit-base-patch16")

# Sample PDF content (text and images) as input
# pdf_text = "This is the text content extracted from the PDF."
# pdf_images = [image_data1, image_data2]  # Image data as torch tensors

# Sample question
question = "What is depicted in the second image?"

# Encode text and images
text_tokens = tokenizer(pdf_text, return_tensors="pt", padding=True, truncation=True)
image_tensors = torch.stack(pdf_images)

# Pass the encoded data to the Oscar model for understanding
with torch.no_grad():
    outputs = model(**text_tokens, pixel_values=image_tensors)

# Prepare the question for Oscar
question_inputs = tokenizer(question, return_tensors="pt")

# Generate answers to the question
answer = model.get_answers(question_inputs, outputs["pixel_values"])

# Print the answer
print("Question:", question)
print("Answer:", answer["text"])
